In [1]:
import numpy as np
import pandas as pd

In [2]:
adamsmith_df = pd.read_pickle('repos/adamsmith/adamsmith_game-theory-poker_pr.pk1')
embulk_df = pd.read_pickle('repos/embulk/embulk_embulk_pr.pk1')
google_df = pd.read_pickle('repos/google/google_iosched_pr.pk1')
guillaume_df = pd.read_pickle('repos/guillaumebort/guillaumebort_play20-spring-demo_pr.pk1')
iven_df = pd.read_pickle('repos/iven-he/iven-he_webspider_pr.pk1')
microsoft_df = pd.read_pickle('repos/microsoft/microsoft_Mesh-processing-library_pr.pk1')
oleiade_df = pd.read_pickle('repos/oleiade/oleiade_trousseau_pr.pk1')
pandas_df = pd.read_pickle('repos/pandas-dev/df_pr_pandas.pk1')
yang_df = pd.read_pickle('repos/yangyangwithgnu/yangyangwithgnu_hardseed_pr.pk1')
zhn_df = pd.read_pickle('repos/zhnnnnn/zhnnnnn_ZHNCosmos_pr.pk1')

df_list = [adamsmith_df, embulk_df, google_df,
           guillaume_df, iven_df, microsoft_df,
           oleiade_df, pandas_df, yang_df, zhn_df]

In [3]:
for df in df_list:
    print(df.shape)

(1, 12)
(786, 12)
(134, 12)
(3, 12)
(8, 12)
(0, 0)
(46, 12)
(12405, 11)
(12, 12)
(0, 0)


In [4]:
embulk_df.head()

,author,authorAssociation,bodyText,closedAt,comments,createdAt,files,mergedBy,resourcePath,state,title,updatedAt
0,"{'login': 'frsyuki', 'company': 'Treasure Data...",CONTRIBUTOR,Implemented recovery from multiline quoted bro...,2014-12-03T20:45:18Z,"{'totalCount': 1, 'nodes': [{'author': {'login...",2014-12-03T03:31:35Z,{'totalCount': 3},{'login': 'muga'},/embulk/embulk/pull/1,MERGED,CsvTokenizer,2014-12-12T23:28:36Z
1,"{'login': 'muga', 'company': 'Treasure Data, I...",CONTRIBUTOR,,2015-01-19T03:59:26Z,"{'totalCount': 1, 'nodes': [{'author': {'login...",2015-01-16T06:25:49Z,{'totalCount': 1},{'login': 'muga'},/embulk/embulk/pull/2,MERGED,inserted logging for showing the progress of e...,2015-02-01T12:24:56Z
2,"{'login': 'muga', 'company': 'Treasure Data, I...",CONTRIBUTOR,,2015-01-20T19:09:29Z,"{'totalCount': 1, 'nodes': [{'author': {'login...",2015-01-16T19:39:52Z,{'totalCount': 1},{'login': 'muga'},/embulk/embulk/pull/3,MERGED,inserted logging for noticing users skipped re...,2015-02-01T12:24:26Z
3,"{'login': 'muga', 'company': 'Treasure Data, I...",CONTRIBUTOR,format strftime strings in path prefixes with ...,2015-01-24T11:52:26Z,"{'totalCount': 2, 'nodes': [{'author': {'login...",2015-01-17T11:06:02Z,{'totalCount': 96},{'login': 'frsyuki'},/embulk/embulk/pull/4,MERGED,added formatting strftime string in S3 path pr...,2015-02-01T12:24:26Z
4,"{'login': 'muga', 'company': 'Treasure Data, I...",CONTRIBUTOR,,2015-01-26T19:59:10Z,"{'totalCount': 5, 'nodes': [{'author': {'login...",2015-01-24T02:15:36Z,{'totalCount': 113},None,/embulk/embulk/pull/6,CLOSED,Add gradle settings,2015-02-01T12:24:28Z


In [5]:
def clean_pr_df(pr_df: pd.DataFrame) -> pd.DataFrame:
    """Takes a DataFrame of issue data and returns a cleaned DataFrame."""

    author_dict_list = pr_df['author'].values.tolist()
    unpacked_dict_list = []

    for item in author_dict_list:
        # If `item` is empty append dict of NaNs, else append data
        if item == None:
            unpacked_dict_list.append({'login': np.NaN, 'company': np.NaN})
        else:
            unpacked_dict_list.append(item)
    
    # Recombine issue_df and unpacked dicts
    author_pr_df = pd.DataFrame(unpacked_dict_list)
    clean_pr_df = pr_df.drop(labels=['author'], axis='columns')
    clean_pr_df['author'] = author_pr_df['login']
    clean_pr_df['company'] = author_pr_df['company']

    # Unpack `files` column and `mergedBy` column
    files_dict_list = pr_df['files'].values.tolist()
    unpacked_dict_list = []
    
    for item in files_dict_list:
        # If `item` is empty append dict of NaNs, else append data
        if item == None:
            unpacked_dict_list.append({'totalCount': np.NaN})
        else:
            unpacked_dict_list.append(item)
    
    files_pr_df = pd.DataFrame(unpacked_dict_list)
    clean_pr_df = clean_pr_df.drop(labels=['files'], axis='columns')
    clean_pr_df['files'] = files_pr_df
    
    mergedBy_dict_list = pr_df['mergedBy'].values.tolist()
    unpacked_dict_list = []
    
    for item in mergedBy_dict_list:
        # If `item` is empty append dict of NaNs, else append data
        if item == None:
            unpacked_dict_list.append({'login': np.NaN})
        else:
            unpacked_dict_list.append(item)
    
    mergedBy_pr_df = pd.DataFrame(unpacked_dict_list)
    clean_pr_df = clean_pr_df.drop(labels=['mergedBy'], axis='columns')
    clean_pr_df['mergedBy'] = mergedBy_pr_df

    
    # Type conversion
    try:
        clean_pr_df['createdAt'] = clean_pr_df['createdAt'].apply(pd.Timestamp)
        clean_pr_df['updatedAt'] = clean_pr_df['updatedAt'].apply(pd.Timestamp)
        clean_pr_df['closedAt'] = clean_pr_df['closedAt'].apply(pd.Timestamp)
    except:
        pass
    return clean_pr_df

In [6]:
adamsmith_df = clean_pr_df(adamsmith_df)
embulk_df = clean_pr_df(embulk_df)
google_df = clean_pr_df(google_df)
guillaume_df = clean_pr_df(guillaume_df)
iven_df = clean_pr_df(iven_df)
# microsoft_df = clean_pr_df(microsoft_df)
oleiade_df = clean_pr_df(oleiade_df)
pandas_df = clean_pr_df(pandas_df)
yang_df = clean_pr_df(yang_df)
# zhn_df = clean_pr_df(zhn_df)


adamsmith_df.to_pickle('./repos/adamsmith_pr_cleaned.pk1')
embulk_df.to_pickle('./repos/embulk_pr_cleaned.pk1')
google_df.to_pickle('./repos/google_pr_cleaned.pk1')
guillaume_df.to_pickle('./repos/guillaume_pr_cleaned.pk1')
iven_df.to_pickle('./repos/iven_pr_cleaned.pk1')
# microsoft_df.to_pickle('./repos/microsoft_pr_cleaned.pk1')
oleiade_df.to_pickle('./repos/oleiade_pr_cleaned.pk1')
pandas_df.to_pickle('./repos/pandas_pr_cleaned.pk1')
yang_df.to_pickle('./repos/yang_pr_cleaned.pk1')
# zhn_df.to_pickle('./repos/zhn_pr_cleaned.pk1')